In [1]:
#####Note:run in python2######
# same initialization of weights because same random seed used

#SGD has

import math
import numpy as np
import sys
from PIL import Image
import matplotlib.pyplot as plt 

#for shuffling data
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)

In [2]:
#########
#MLP

# initialisation of the weights he_normal
def weights(noHiddenLayers,sizeOfLayers):

    W=[]
    b=[]

    for i in range(0,noHiddenLayers+1):
        
        W.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+sizeOfLayers[i])),
                                   (sizeOfLayers[i+1],sizeOfLayers[i])) )
        b.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+1)),
                                   (sizeOfLayers[i+1],1)) )

    W=np.array(W)
    b=np.array(b)
    
    return W,b


#mlp forward pass
#layer
def layer(w,x,b):
    out = np.matmul(w,x)+b
    return out

def apply_activationMLP(Activation_function,inp):
    
    #activation functions
    if Activation_function == 'relu':
        return np.where(inp<0,0,inp)
    elif Activation_function == "tanh":
        return np.tanh(inp)
    elif Activation_function == "sigmoid":
        return 1.0/(1+np.exp(-1.0*inp))
    elif Activation_function == "softmax":
        return (1.0/(np.sum(np.exp(inp),axis=0)))*(np.exp(inp))

#forward path
def forward_path(noHiddenLayers,X,W,b,Actfnvect):

    out=[]
    
    z=apply_activationMLP(Actfnvect[0],np.array(layer(W[0],X,b[0])))
    out.append(np.array(z))

    for i in range(1,noHiddenLayers):
        z=apply_activationMLP(Actfnvect[i],np.array(layer(W[i],out[i-1],b[i])))
        out.append(np.array(z))

    if noHiddenLayers > 0:
        z=apply_activationMLP(Actfnvect[-1],np.array(layer(W[-1],out[-1],b[-1])))
        out.append(np.array(z))

    y_pred = out[-1]

    return np.array(out),np.array(y_pred)


In [3]:
#only to import data
from keras import backend as K
import keras
from keras.datasets import mnist


# #import data
iris_data = load_iris() # load the iris dataset

x = iris_data.data
y_ = iris_data.target.reshape(-1, 1) # Convert data to a single column

# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
# Split the data for training and testing
x_train , x_test, y_train , y_test = train_test_split(x, y, test_size=0.20)

print(x_train.shape)
print(y_train.shape)

Using TensorFlow backend.


(120, 4)
(120, 3)


In [4]:
# run MLP algorithm

x_train = np.moveaxis(x_train,0,-1)
y_train = np.moveaxis(y_train,0,-1)
x_test = np.moveaxis(x_test,0,-1)
y_test = np.moveaxis(y_test,0,-1)

print x_train.shape
print y_train.shape
print x_test.shape
print y_test.shape

(4, 120)
(3, 120)
(4, 30)
(3, 30)


In [5]:
#derivative of relu
def der_relu(x):
    return np.where(x == 0,0,1)

# backpropation
def backprop(y,y_true,z,T,u,M,x):
    
    dy = (y-y_true)
    
    # layer 3
    dT  = np.matmul(dy,z.T)
    db3 = np.sum(dy,axis=1).reshape(-1,1)
    
    # layer 2
    s   = np.matmul(T.T,dy)
    s   = s*der_relu(z)
    dM  = np.matmul(s,u.T)
    db2 = np.sum(s,axis=1).reshape(-1,1)
    
    # layer 1
    sm  = np.matmul(M.T,s)
    sm  = sm*der_relu(u)
    dW  = np.matmul(sm,x.T)
    db1 = np.sum(sm,axis=1).reshape(-1,1)
    
    return dW,dM,dT,db1,db2,db3

In [6]:
###
################################
# Training parameters
num_classes = 3
epochs = 600
learning_rate=0.0001
batch_size = 10
alpha = 0.9
################################

#MLP PARAMETERS
noHiddenLayers=2

#also includes the input vector dimension and output vector dimension
sizeOfLayers=[x_train.shape[0],10,10,num_classes]

sizeofOutput=num_classes

Actfnvect = ['relu','relu','softmax']

In [7]:
W,b = weights(noHiddenLayers,sizeOfLayers)

In [8]:
# (a)
# Momentum descent

# updating the weights


v = {"W[0]" : np.zeros(W[0].shape) , "W[1]" : np.zeros(W[1].shape) ,"W[2]" : np.zeros(W[2].shape),"b[0]": np.zeros(b[0].shape)
     ,"b[1]" :np.zeros(b[1].shape) , "b[2]" : np.zeros(b[2].shape)}

for i in range(epochs):
    loss=0
    for j in np.arange(0,x_train.shape[1],batch_size):       
        dW = np.zeros(W[0].shape)
        dM = np.zeros(W[1].shape)
        dT = np.zeros(W[2].shape)
        db1 = np.zeros(b[0].shape)
        db2 = np.zeros(b[1].shape)
        db3 = np.zeros(b[2].shape)
        for k in range(0,batch_size):
            # forward pass
            x = x_train[:,j+k].reshape(-1,1)
            y = y_train[:,j+k].reshape(-1,1)
            out,y_pred=forward_path(noHiddenLayers,x,W,b,Actfnvect)
            # backpropagation
            dWtemp,dMtemp,dTtemp,db1temp,db2temp,db3temp=backprop(y_pred,y,out[1],W[-1],out[0],W[1],x)
            
            dW=dW+dWtemp
            db1=db1+db1temp
            
            dM=dM+dMtemp
            db2=db2+db2temp
            
            dT=dT+dTtemp
            db3=db3+db3temp
        
            # calculate the loss
            loss = loss + (-1.0*np.sum(y*np.log(y_pred)))
            
        # Updating the weights using MOMENTUM Approach
        
        dW=dW*(1.0/batch_size)
        dM=dM*(1.0/batch_size)
        dT=dT*(1.0/batch_size)
        db1=db1*(1.0/batch_size)
        db2=db2*(1.0/batch_size)
        db3=db3*(1.0/batch_size)
        
        v["W[0]"] = alpha*v["W[0]"] - learning_rate*(dW)
        v["W[1]"] = alpha*v["W[1]"] - learning_rate*(dM)
        v["W[2]"] = alpha*v["W[2]"] - learning_rate*(dT)
        v["b[0]"] = alpha*v["b[0]"] - learning_rate*(db1)
        v["b[1]"] = alpha*v["b[1]"] - learning_rate*(db2)
        v["b[2]"] = alpha*v["b[2]"] - learning_rate*(db3)
        
        W[0] = W[0] + v["W[0]"]
        W[1] = W[1] + v["W[1]"]
        W[2] = W[2] + v["W[2]"]
        b[0] = b[0] + v["b[0]"]
        b[1] = b[1] + v["b[1]"]
        b[2] = b[2] + v["b[2]"]
        
    
    #print the loss in each epoch
    print('Epoch:'+str(i)+'         Loss:'+str(loss))

Epoch:0         Loss:257.79156193871694
Epoch:1         Loss:240.90412559731556
Epoch:2         Loss:222.69811766059388
Epoch:3         Loss:207.4910184586755
Epoch:4         Loss:195.5050476557887
Epoch:5         Loss:186.02023232454906
Epoch:6         Loss:178.38858420714206
Epoch:7         Loss:172.05413158964694
Epoch:8         Loss:166.61362488404038
Epoch:9         Loss:161.8280765073398
Epoch:10         Loss:157.5536642874321
Epoch:11         Loss:153.7140073719533
Epoch:12         Loss:150.37153645401557
Epoch:13         Loss:147.36929820382213
Epoch:14         Loss:144.55448733559143
Epoch:15         Loss:141.69241160108237
Epoch:16         Loss:138.65085032406907
Epoch:17         Loss:136.0144766046348
Epoch:18         Loss:133.6901196716816
Epoch:19         Loss:131.49217324561207
Epoch:20         Loss:129.35582962309476
Epoch:21         Loss:127.24839858599891
Epoch:22         Loss:125.15234246712328
Epoch:23         Loss:123.07183441223826
Epoch:24         Loss:120.9899501

Epoch:208         Loss:34.589217654215034
Epoch:209         Loss:34.457676467429046
Epoch:210         Loss:34.32699737606945
Epoch:211         Loss:34.1959291062826
Epoch:212         Loss:34.06625429321151
Epoch:213         Loss:33.937544428558375
Epoch:214         Loss:33.80967001896952
Epoch:215         Loss:33.68312756736829
Epoch:216         Loss:33.55619897451095
Epoch:217         Loss:33.43051812085814
Epoch:218         Loss:33.30478593397724
Epoch:219         Loss:33.17996131482233
Epoch:220         Loss:33.05611277483098
Epoch:221         Loss:32.9324507580483
Epoch:222         Loss:32.80997417451107
Epoch:223         Loss:32.68732146358726
Epoch:224         Loss:32.565638404469155
Epoch:225         Loss:32.445127646416616
Epoch:226         Loss:32.32441065832353
Epoch:227         Loss:32.204671565496156
Epoch:228         Loss:32.08600080043926
Epoch:229         Loss:31.967107209277142
Epoch:230         Loss:31.849091757715065
Epoch:231         Loss:31.732435117721
Epoch:232   

Epoch:417         Loss:17.85957716632973
Epoch:418         Loss:17.81828404200489
Epoch:419         Loss:17.777339286116707
Epoch:420         Loss:17.736658474192186
Epoch:421         Loss:17.696217568023496
Epoch:422         Loss:17.655969473694125
Epoch:423         Loss:17.615934416082446
Epoch:424         Loss:17.576132257220745
Epoch:425         Loss:17.536453589897807
Epoch:426         Loss:17.497185438759082
Epoch:427         Loss:17.458126628632513
Epoch:428         Loss:17.41903441713444
Epoch:429         Loss:17.380220237044107
Epoch:430         Loss:17.341846010037667
Epoch:431         Loss:17.30366715643588
Epoch:432         Loss:17.265406205029883
Epoch:433         Loss:17.227545321338624
Epoch:434         Loss:17.190070523629938
Epoch:435         Loss:17.152507488903638
Epoch:436         Loss:17.115222144777036
Epoch:437         Loss:17.0782343242016
Epoch:438         Loss:17.041395544285862
Epoch:439         Loss:17.004887293430414
Epoch:440         Loss:16.96831606772147

In [9]:
out,y_pred=forward_path(noHiddenLayers,x_test,W,b,Actfnvect)

In [10]:
def predict(y):
    return np.argmax(y)

yvect=[]
y_trurevect=[]

for i in range(0,x_test.shape[1]):
    yvect.append(predict(y_pred[:,i]))
    y_trurevect.append(predict(y_test[:,i]))

# find accuracy
from sklearn.metrics import accuracy_score
#predicting test accuracy
print(accuracy_score(y_trurevect, yvect))

1.0


In [11]:
# to see the output vs true values

print y_trurevect
print yvect

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
